### Import libraries

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse

import csv
import os
import pandas as pd

import time

#### Executing Selenium

In [1]:
def executing_selenium(url):
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(2)
    return driver

#### Xử lí url

In [ ]:
def remove_last_slash(link):
    # Kiểm tra xem đường dẫn có kết thúc bằng dấu "/" không
    if link.endswith("/"):
        # Nếu có, loại bỏ dấu "/" cuối cùng bằng cách cắt chuỗi từ đầu đến ký tự thứ hai từ cuối
        remove_link = link[:-1]
        return remove_link
    return link

In [ ]:
def remove_first_slash(link):
    if link.startswith("/"):
        remove_link = link[1:]
        return remove_link
    return link

print(remove_first_slash("/may-tinh-bang"))

may-tinh-bang


In [ ]:
def extract_path_and_remove_query_params(url):
    parsed_url = urlparse(url)
    path_without_query = parsed_url.path.split('?')[0]
    path_parts = path_without_query.split('/')
    category = path_parts[1]
    product_name = path_parts[2]
    return category, product_name

In [ ]:
def split_url(url):
    # Sử dụng biểu thức chính quy để tìm vị trí của kí tự số cuối cùng
    match = re.search(r'\d+$', url)
    if match:
        # Nếu tìm thấy, lấy vị trí của kí tự số cuối cùng
        poi = match.start()
        # Tách chuỗi thành hai phần
        prefix = url[:poi]
        endfix = url[poi:]
        return prefix, endfix
    else:
        # Nếu không tìm thấy kí tự số cuối cùng, trả về None cho cả hai phần
        return url, ""

#### Check categories nào nằm trong danh sách crawl

In [ ]:
def check_keywords_in_link(link, keywords):
    keyword_pattern = re.compile('|'.join(keywords), re.IGNORECASE)
    return bool(keyword_pattern.search(link))

#### Crawl category

In [ ]:
def crawl_category(base_link, categories, remove_key_word):
    driver = executing_selenium(base_link)
    content = driver.page_source
    result = []
    soup = BeautifulSoup(content, "html.parser")
    menu = soup.find('ul', class_='main-menu')
    list_a =  menu.findChildren('a')
    base_link_remove_last_slash = remove_last_slash(base_link)
    for i in list_a:
        if not any (word in i.get('href') for word in remove_key_word) and i.get('href') != None and check_keywords_in_link(i.get('href'), categories):
            cate_link = base_link_remove_last_slash + i.get('href')
            result.append([remove_first_slash(i.get('href')), cate_link])
            print([remove_first_slash(i.get('href')), cate_link])

    return result

#### Crawl max page of category

In [ ]:
def get_end_page_from_category(url):
    #first find url
    driver = executing_selenium(url)
    category_url = ""
    try:
        driver.find_element(By.CLASS_NAME,'view-more')
        driver.find_element(By.CLASS_NAME,'view-more > a').click()
        time.sleep(2)
        category_url = driver.current_url
    except:
        category_url = url

    pre, end = split_url(category_url)
    if pre != "" and end != "":
        stop = False
        end_url = pre
        count = int(end)
        while stop == False:
            driver.get(end_url + str(count))
            try:
                driver.find_element(By.CLASS_NAME, 'view-more')
                driver.find_element(By.CLASS_NAME,'view-more > a').click()
                time.sleep(2)
                count += 1
                end_url = driver.current_url
            except:
                stop = True

        end = count
        print(pre + str(end))
    return pre + str(end)

In [ ]:
def crawl_list(base_link, url):
    driver =  executing_selenium(url)
    content = driver.page_source
    remove_key_words = ['javascript:void(0)','#']
    list_href = []
    soup = BeautifulSoup(content, "html.parser")

    list_products = soup.find('ul', class_='listproduct')
    base_link_remove_last_slash = remove_last_slash(base_link)
    if list_products != None:
        li_products = list_products.findChildren('li')
        for li in li_products:
            a_products = li.findChildren('a')
            for a in a_products:
                if not any (word in a.get('href') for word in remove_key_words):
                    product_url = base_link_remove_last_slash + a.get('href')
                    list_href.append([a.get("data-name"), a.get('href'), product_url])

        print(f"Get {len(li_products)} from {url}")

    return list_href

#### Save Data

In [ ]:
# def write_to_csv(data, path, column_names, index = True, append=False):
#     if data == None:
#         return "Data is null"
#     if index == True:
#         #Đánh index cho mỗi record
#         data = [(i+1, item) for i, item in enumerate(data)]
#     # check xem có phải là chế độ ghi liên tiếp hay không
#     if append:
#         # Tồn tại file đó hay không
#         if not os.path.exists(path):
#             # Nếu tập tin chưa tồn tại, tạo DataFrame từ dữ liệu và ghi vào tập tin
#             df = pd.DataFrame(data, columns=column_names)
#             df.to_csv(path, index=False)
#             return "Data has written to csv file"
#         df = pd.DataFrame(data)
#         df.to_csv(path, mode='a', header=False, index=False)

#     else:
#         if not os.path.exists(path):
#             # Nếu tập tin chưa tồn tại, tạo DataFrame từ dữ liệu và ghi vào tập tin
#             df = pd.DataFrame(data, columns=column_names)
#             df.to_csv(path, index=False)
#             return "Data has written to csv file"
#         else:
#             os.remove(path)
#             print ("Found a csv file, deleting it...")
#             df = pd.DataFrame(data, columns=column_names)
#             df.to_csv(path, index=False)
#             return "Data has written to csv file"


import os
import pandas as pd

def write_to_csv(data, path, column_names, index=True, append=False):
    if data is None or len(data) == 0:
        return "Data is null or empty"

    # Gắn index nếu cần
    if index:
        data = [(i + 1, *item) for i, item in enumerate(data)]
        column_names = ["index"] + column_names

    # Chuyển data thành DataFrame
    df = pd.DataFrame(data, columns=column_names)

    # Ghi tiếp vào file
    if append and os.path.exists(path):
        df.to_csv(path, mode='a', header=False, index=False)
        return f"Appended {len(df)} rows to {path}"
    else:
        # Ghi đè (hoặc lần đầu tạo)
        df.to_csv(path, index=False)
        return f"Wrote {len(df)} rows to {path}"


#### Read Data

In [ ]:
def read_csv(path, is_read_header = True):
    data = []
    if not os.path.exists(path):
        print(f"File '{path}' does not exist.")
        return data
    # Mở tập tin CSV để đọc
    with open(path, 'r', newline='',encoding='utf-8') as file:
        reader = csv.reader(file)
        if is_read_header:
            next(reader)
        # Duyệt qua từng hàng trong tập tin CSV
        for row in reader:
            data.append(row)
    return data

#### Main
##### 1. Crawl all category of thegioididong and save to csv file
##### 2. Crawl all end page (find button view-more at the end of the page')
##### 3. Crawl all product link from end page link

In [ ]:
base_link = "https://www.thegioididong.com/"

removing_keyword = ['op-lung', 'mieng-dan', 'ldp', 'tien-ich/', 'gia-do-dien-thoai', 'tui-dung-airpods', 'tui-chong-soc', 'sim-so-dep']
categories = ['dtdd'] # , 'may-tinh-bang', 'laptop', 'phu-kien', 'dong-ho-thong-minh', 'phu-kien-dien-thoai', 'phu-kien-may-tinh-bang', 'phu-kien-laptop']
# # Category crawl
categories = crawl_category(base_link, categories,removing_keyword)
write_to_csv(categories, "tgdt_categories.csv",["category", "full link"], False)

['dtdd', 'https://www.thegioididong.com/dtdd']
['sac-dtdd', 'https://www.thegioididong.com/sac-dtdd']


C:\Users\quang\AppData\Local\Temp\ipykernel_7404\2867265181.py:7: DeprecationWarning: Call to deprecated method findChildren. (Replaced by find_all) -- Deprecated since version 3.0.0.
  list_a =  menu.findChildren('a')


'Wrote 2 rows to tgdt_categories.csv'

In [ ]:
# end_link crawl
end_page_link = []
for i in  read_csv("tgdt_categories.csv", True):
    end_page_link.append([i[0], get_end_page_from_category(i[1])])
print(write_to_csv(end_page_link, "tgdd_end_page_link.csv", ["category", "full link"], False))

https://www.thegioididong.com/dtdd#c=42&o=13&pi=6
https://www.thegioididong.com/sac-dtdd#c=57&o=13&pi=3
Wrote 2 rows to tgdd_end_page_link.csv


In [ ]:
# # product link crawl
for i in read_csv("tgdd_end_page_link.csv", True):
     data = crawl_list(base_link, i[1])
     write_to_csv(data, "tgdd_product.csv", ["name", "category", "Link"], False, True)


C:\Users\quang\AppData\Local\Temp\ipykernel_7404\337275777.py:11: DeprecationWarning: Call to deprecated method findChildren. (Replaced by find_all) -- Deprecated since version 3.0.0.
  li_products = list_products.findChildren('li')
C:\Users\quang\AppData\Local\Temp\ipykernel_7404\337275777.py:13: DeprecationWarning: Call to deprecated method findChildren. (Replaced by find_all) -- Deprecated since version 3.0.0.
  a_products = li.findChildren('a')


Get 292 from https://www.thegioididong.com/dtdd#c=42&o=13&pi=6
Get 80 from https://www.thegioididong.com/sac-dtdd#c=57&o=13&pi=3


In [ ]:
pattern = '[^a-z0-9A-Z_ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂưăạảấầẩẫậắằẳẵặẹẻẽềềểỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễếệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳýỵỷỹ]+'
def preprocess_comment(comment):
    comment = comment.strip()
    comment = comment.replace('  ', ' ');
    return re.sub(pattern,' ', comment).lower()

In [ ]:
def crawl_comment(base_url, category, product):
    data = []
    count = 0

    url = base_url + category + '/' + product
    print(url)
    driver = executing_selenium(url)
    time.sleep(1)
    try:
        driver.find_element(By.CLASS_NAME, 'btn-view-all')
        url = base_url + category + '/' + product + '/danh-gia?page='
        number_page = 1
        while True:
            driver = executing_selenium(url + str(number_page))
            time.sleep(1)
            content = driver.page_source
            soup = BeautifulSoup(content, "html.parser")
            ul_comments = soup.find('ul', class_='comment-list')
            if ul_comments is None:
                print(f"Crawl {base_url + category + '/' + product } done!! Found {count} comments")
                break
            li_comments = ul_comments.find_all('li', {"class" : "par"})
            for li_comment in li_comments:
                div_comment = li_comment.find('div', {"class": "cmt-content"})
                start_buy = li_comment.find_all('i', {'class' : 'iconcmt-starbuy'})
                comment = preprocess_comment(div_comment.text)
                if comment != "":
                    data.append([category, product, len(start_buy), comment])
                    count += 1
            # div_comments = ul_comments.find_all('div', {"class": "cmt-content"})
            # for div_comment in div_comments:
            #     comment = preprocess_comment(div_comment.text)
            #     if comment != "":
            #         data.append([category, product, comment])
            #         count += 1
            number_page = number_page + 1
    except:
        content = driver.page_source
        soup = BeautifulSoup(content, "html.parser")
        ul_comments = soup.find('ul', class_='comment-list')
        if ul_comments is None:
            print(f"Crawl {base_url + category + '/' + product } done!! Found {count} comments")
        else:
            # div_comments = ul_comments.find_all('p', {"class": "cmt-txt"})
            # for div_comment in div_comments:
            #     comment = preprocess_comment(div_comment.text)
            #     if comment != "":
            #         data.append([category, product, comment])
            #         count += 1
            li_comments = ul_comments.find_all('li', class_='par')
            for li_comment in li_comments:
                div_comment = li_comment.find('div', {"class": "cmt-tx"})
                start_buy = li_comment.find_all('i', {'class' : 'iconcmt-starbuy'})
                comment = preprocess_comment(div_comment.text)
                if comment != "":
                    data.append([category, product, len(start_buy), comment])
                    count += 1

    return data

In [ ]:
with open('product.csv', mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ["category", "product", "star", 'comments']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
print("Đã tạo file product.csv thành công.")
products = read_csv('product.csv', True)
for i in products:
    category, product_name = extract_path_and_remove_query_params(i[1])
    comments = crawl_comment(base_link, category, product_name)
    write_to_csv(comments, "tgdd_comments.csv", ["category", "product", "star",'comments'], False, True)
comments = crawl_comment(base_link, "dtdd", "samsung-galaxy-s24-ultra-5g")
write_to_csv(comments, "product.csv", ["category", "product","star", 'comments'], False, True)

Đã tạo file product.csv thành công.
https://www.thegioididong.com/dtdd/samsung-galaxy-s24-ultra-5g
Crawl https://www.thegioididong.com/dtdd/samsung-galaxy-s24-ultra-5g done!! Found 130 comments


'Appended 130 rows to product.csv'

In [ ]:
# Nếu có lỗi trong quá trình crawl thì chạy hàm này
def crawl_product_continue(current_product):
    product_list = read_csv('product.csv', True)
    df = pd.read_csv('product.csv')
    product_category = df["category"].tolist()
    try:
        target_index = product_category.index(current_product)
        products_after_target = product_list[target_index:]
        for i in products_after_target:
            category_continue, product_name_continue = extract_path_and_remove_query_params(i[1])
            comments = crawl_comment(base_link, category_continue, product_name_continue)
            write_to_csv(comments, "tgdd_comments.csv", ["category", "product", "star",'comments'], False, True)
        return products_after_target
    except ValueError:
        print("không tìm thấy sản phẩm")
        return []

crawl_product_continue("/phan-mem/eset-nod32-antivirus-cho-windows-1-pc")

không tìm thấy sản phẩm


[]